# Практикум Python


<img src="https://www.python.org/static/community_logos/python-logo-master-v3-TM.png" align="right" style="height: 200px;"/>

# Занятие 11. Оптимизация кода. NumPy. Cython


Медлительность Python по сравнению с другими ЯП - известный печальный факт, который отталкивает многих от использования его в реальных проектах, где скорость критична.

Но во многих случаях Python можно ускорить (при чем значительно)! Рассмотрим как это можно сделать на примере класса `Matrix`:

In [ ]:
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        return cls([[0] * shape[1] for i in range(shape[0])])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M
    
    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return ((0, 0) if not self else
                (len(self), len(self[0])))

Жизненно необходимая функция - умножение двух матриц. Реализуем ее самым простым образом:

In [ ]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    
    assert n_xcols == n_yrows, "Incompatible matrix dimensions"
    
    Z = Matrix.zeros((n_xrows, n_ycols))
    
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z

Проверим, что работает корректно:

In [ ]:
X = Matrix([[1], 
            [2], 
            [3]])

Y = Matrix([[4, 5, 6]])

print(matrix_product(X, Y))
print(matrix_product(Y, X))

[[4, 5, 6], [8, 10, 12], [12, 15, 18]]
[[32]]


Посмотрим, сколько будет работать такое умножение:

In [ ]:
def setup(shape=(100, 100)):
    X = Matrix.random(shape)
    Y = Matrix.random(shape)
    return X, Y

In [ ]:
X_test, Y_test = setup()

In [ ]:
%%timeit
matrix_product(X_test, Y_test)

351 ms ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Это очень много! Посмотрим, где именно мы теряем время, для этого можно установить `line_profiler`:

In [ ]:
!pip install line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [ ]:
%lprun -f matrix_product matrix_product(*setup())

Видим, что куча времени уходит на `for` и на доставание индексов. Как думаете, что можно улучшить?

# Базовая оптимизация

In [ ]:
def matrix_product2(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    
    for i in range(n_xrows):
        Xi = X[i]
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += Xi[j] * Y[j][k]
    return Z

In [ ]:
%%timeit
matrix_product2(X_test, Y_test)

302 ms ± 6.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Попробуем убрать лишнее доставание элемента по индексу:

In [ ]:
def matrix_product3(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    
    for i in range(n_xrows):
        Xi = X[i]
        for k in range(n_ycols):
            acc = 0
            for j in range(n_xcols):
                acc += Xi[j] * Y[j][k]
            Z[i][k] = acc
    return Z

In [ ]:
%%timeit
matrix_product3(X_test, Y_test)

193 ms ± 4.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Стало почти в 2 раза быстрее! Так как стало примерно в 2 раза меньше операций доставания по индексу.

In [ ]:
%lprun -f matrix_product3 matrix_product3(*setup())

Пойдём дальше и попробуем ещё ускориться

In [ ]:
def matrix_product4(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()  # :)
    
    for i, (Xi, Zi) in enumerate(zip(X, Z)):
        for k, Ytk in enumerate(Yt):
            Zi[k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z

In [ ]:
%%timeit
matrix_product4(X_test, Y_test)

143 ms ± 6.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Стало ещё в 1.5 раза быстрее!

**Вывод:** если ваш код тормозит, не спешите использовать другие фреймворки. Часто можно ускорить в разы свой код небольшими манипуляциями.

# NumPy

**[NumPy]((http://www.numpy.org/))** (**Num**eric **Py**thon) - это open-source модуль для Python, который предоставляет общие математические и числовые операции в виде пре-скомпилированных, быстрых функций (использует типы из C, которые существенно быстрее чем Python типы). Они обеспечивают функционал, который можно сравнить с функционалом MatLab.

In [ ]:
import numpy as np

**Преимущества NumPy:**

- **Быстрее** - NumPy использует алгоритмы, написанные на C, которые работают *значительно быстрее* чем те, что написаны на Python.
- **Меньше циклов** - NumPy помогает уменьшить количество циклов в коде вашей программы, что мешает не запутаться в них.
- **Чище код** - без вложенных циклов ваш код больше будет намного чище и проще для чтения.
- **Better quality** - код библиотеки написан сообществом профессионалов, поэтому он оптимизирован, приятен в работе и не содержит багов.

По этим причинам NumPy является де-факто стандартом для работы с многомерными массивами в анализе данных. Многие популярные библиотеки в своем исходном коде используют NumPy.

Попробуем взять нашу первую самую тупую реализацию, но использовать не наш класс `Matrix`, а реализацию массива из `numpy` (позже посмотрим, как она устроена):

In [ ]:
import numpy as np

def np_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [ ]:
def setup_np(shape=(100, 100)):
    X = np.random.randint(-255, 255, size=shape)
    Y = np.random.randint(-255, 255, size=shape)
    return X, Y

In [ ]:
%timeit np_matrix_product(*setup_np())

702 ms ± 13.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Никакого профита пока нет. Попробуем вместо нашей функции перемножения матриц воспользоваться функцией самого `numpy`:

In [ ]:
X, Y = setup_np()

In [ ]:
%timeit np.dot(X, Y)

1.08 ms ± 60.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


А вот это уже интересно. Достойная мотивация для изучения модуля `numpy`.

## Array

Основным объектом `NumPy` является *однородный* многомерный массив, в numpy он реализован через объект `ndarray`. Массивы (`ndarray`) похожи на списки (`list`), но могут хранить только элементы одного типа. Производить вычисления с массивами гораздо быстрее и эффективнее чем со списками.

In [ ]:
a = np.array([4, 1.1, 2, 3, True], dtype=np.int32)

In [ ]:
a[1] = 5

In [ ]:
a.size

5

In [ ]:
a.shape

(5,)

In [ ]:
a.dtype

dtype('int32')

In [ ]:
a

array([4, 5, 2, 3, 1], dtype=int32)

In [ ]:
type(a)

numpy.ndarray

Массив имеет один тип. Если при создании передать разные типы, то он всё равно всё приведёт к одному типу.

In [ ]:
np.array([4, 1.1, 2, 3, "string"])

array(['4', '1.1', '2', '3', 'string'], dtype='<U32')

## UFuncs

**UFuncs (universal functions)** - это функции, которые работают над массивом поэлементно.

Рассмотрим на примере простой функции:

In [ ]:
import math

a = range(4)
b = [math.sin(value) for value in a]
print(b)

[0.0, 0.8414709848078965, 0.9092974268256817, 0.1411200080598672]


Такой код будет в numpy выглядеть так:

In [ ]:
a = np.arange(4)
b = np.sin(a)
print(b)

[0.         0.84147098 0.90929743 0.14112001]


Это работает для любых универсальных функций.

Сложение двух массивов происходит поэлементно и при этом быстро:

In [ ]:
count = int(1e7)
a, b = [random.random() for _ in range(count)], [random.random() for _ in range(count)]

%timeit [i + j for i, j in zip(a, b)]

1.09 s ± 14.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
a, b = np.array(a), np.array(b)
%timeit a + b

31.6 ms ± 320 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Это работает быстро, так как всё итерирование протащено в компилируемую часть кода.

Есть и другие UFuncs, т.е. поэлементные функции. Например посмотрим, сколько займёт поэлементное возведение массива в квадрат:

In [ ]:
def slow():
    a = range(10000)
    return [i ** 2 for i in a]

%timeit slow()

2.79 ms ± 60 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
def fast():
    a = np.arange(10000)
    return a ** 2

%timeit fast()

16.1 µs ± 187 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Есть много других UFuncs: арифметические операции, побитовые, сравнения, тригонометрические и т.д.

## Aggregation

В numpy много встроенных агрегационных функций:

**Function Name** | **NaN-safe Version** | **Description**
- | - | -
`np.sum` | `np.nansum` | Compute sum of elements
`np.prod` | `np.nanprod` | Compute product of elements
`np.mean` | `np.nanmean` | Compute mean of elements
`np.std` | `np.nanstd` | Compute standart deviation
`np.var` | `np.nanvar` | Compute variance
`np.min` | `np.nanmin` | Find minimum value
`np.max` | `np.nanmax` | Find maximum value
`np.argmin` | `np.nanargmin` | Find index of minimum value
`np.argmax` | `np.nanargmax` | Find index of maximum value
`np.median` | `np.nanmedian` | Compute median of elements
`np.percentile` | `np.nanpercentile` | Compute rank-based statistics of elements
`np.any` | - | Evaluate whether any elements are true
`np.all` | - | Evaluate whether all elements are true

Сравним с агрегационными функциями, встроенными в Python:

In [ ]:
data = [random.random() for _ in range(1000000)]
%timeit min(data)

16.7 ms ± 697 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
np_data = np.array(data)
%timeit np.min(np_data)

434 µs ± 12.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


**Примечание:** как получить nan?

In [ ]:
float('nan')

nan

In [ ]:
float('Inf') - float('Inf')

nan

In [ ]:
np.inf - np.inf

nan

## Индексация

Есть все базовые срезы и индексы. Но есть немало дополнительных приятных особенностей:

In [ ]:
X = np.arange(6).reshape((2, 3))
X

array([[0, 1, 2],
       [3, 4, 5]])

In [ ]:
X[1]

array([3, 4, 5])

In [ ]:
X[0, 1]

1

In [ ]:
X[:, [0, 2, 1, 2]]

array([[0, 2, 1, 2],
       [3, 5, 4, 5]])

В отличие от обычных списков при слайсах не происходит копирования и можно изменить исходный объект:

In [ ]:
z = X[:1, 1:]
z

array([[1, 2]])

In [ ]:
z[:1, :1] = 100

In [ ]:
X

array([[  0, 100,   2],
       [  3,   4,   5]])

### Маскирование

NumPy поддерживает удобную фичу отбора элементов массива по логическому условию:

In [ ]:
a = np.arange(8) ** 2
a[a > 8]

array([ 9, 16, 25, 36, 49])

Это работает за счёт того, что `numpy.array` поддерживает индексацию по булевой маске:

In [ ]:
a > 8

array([False, False, False,  True,  True,  True,  True,  True])

`True` - берем элемент, `False` - не берем.

## Broadcasting

Мощный механизм, позволяющий NumPy работать с массивами различной формы при выполнении арифметических операций. Часто у нас есть меньший массив и больший массив, и мы хотим использовать меньший массив несколько раз, чтобы выполнить некоторую операцию над большим массивом

[Наглядный туториал](https://scipy.github.io/old-wiki/pages/EricsBroadcastingDoc)

Можно складывать массивы разных размеров там, где это имеет смысл:

In [ ]:
a = np.arange(3)
a

array([0, 1, 2])

In [ ]:
b = np.array(((0, 0, 0), (10, 10, 10), (20, 20, 20), (30, 30, 30)))
b

array([[ 0,  0,  0],
       [10, 10, 10],
       [20, 20, 20],
       [30, 30, 30]])

In [ ]:
b + a

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])

Что по сути происходит - NumPy "расширяет" наш вектор до размера 4x3, а затем выполняет сложение.

<img src="https://i.stack.imgur.com/JcKv1.png" width=900px/>

Broadcasting - не магический, у него свои правила. Поэтому не всегда всё работает так, как хотелось бы:

In [ ]:
X = np.arange(6).reshape((3, 2))
X

array([[0, 1],
       [2, 3],
       [4, 5]])

In [ ]:
X + np.array([1, 2, 3])

ValueError: ignored

In [ ]:
X + np.array([1, 2, 3]).reshape(-1, 1)

array([[1, 2],
       [4, 5],
       [7, 8]])

Мы, конечно, понимаем, что имелось в виду, но numpy не понял. Подскажем ему:

In [ ]:
Y = np.array((1, 2, 3))[:, np.newaxis]  # переориентируем массив
print(Y.shape)
print(Y)

(3, 1)
[[1]
 [2]
 [3]]


In [ ]:
X + Y

array([[1, 2],
       [4, 5],
       [7, 8]])

### Как работает broadcasing?

Broadcasting работает с формами двух массивов:

In [ ]:
a.shape

(3,)

In [ ]:
b.shape

(4, 3)

Их формы представляются следующим образом:

```
b  (2d array): 4 x 3
a  (1d array):     3
```

Первым делом их формы выравниваются с помощью добавления осей-пустышек:

```
b  (2d array): 4 x 3
a  (1d array): 1 x 3
```

Затем мы проходим справа налево и сравниваем количества элементов в двух массивах:

1. Если они одинаковые, то это же количество будет и в результирующем векторе
2. Если один из них равен 1, то другое значение будет в результирующем векторе (растянем по этой оси)
3. Если не равны и ни один из них не равен 1, broadcasting не сработает

Для нашего примера получим следующее:

```
b  (2d array): 4 x 3
a  (1d array): 1 x 3
c  (2d array): 4 x 3
```

# Cython

**Cython**, по своей сути, это промежуточный слой между Python и C/C++. Cython позволяет писать обычный Python-код с некоторыми незначительными модификациями, который затем напрямую транслируется в C-код.

Единственное изменение Python-кода при этом заключается в добавлении к каждой переменной информации об её типе.

## Пример - численное интегрирование

Посмотрим, как он работает. Создадим пару функций на чистом Python:

In [ ]:
def f(x):
    return x**2

def integrate_f_base(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

In [ ]:
%%timeit
integrate_f_base(0, 10, 1_000_000)

224 ms ± 9.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Теперь подключим Cython магию:

In [ ]:
!pip install Cython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [ ]:
%%cython -a

def f(x):
    return x**2


def integrate_f_cython(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

Cython пытается определить тип и в частности за счёт этого ускориться. Но желтым подсвечиваются те строки кода, где он не смог установить тип. Тут почти всё желтое.

In [ ]:
%%timeit
integrate_f_cython(0, 10, 1000000)

117 ms ± 2.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Стало чуть быстрее. Но можно лучше. Применим статическую типизацию к переменным:

In [ ]:
%%cython -a 

def f(double x):
    return x**2


def integrate_f_cython2(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):  # в случае, когда i объевлено как int, этот цикл компилируется в чистый C код
        s += f(a + i * dx)
    return s * dx

In [ ]:
%%timeit
integrate_f_cython2(0, 10, 1000000)

40.8 ms ± 518 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Вызов функций в Python может быть дорогим. Например, в
прошлом примере постоянно происходила конвертация из C
double в Python float. Изменим прототип функции f:

In [ ]:
%%cython -a 

cdef double f_cython(double x):
    return x**2


def integrate_f_cython3(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):  # в случае, когда i объевлено как int, этот цикл компилируется в чистый C код
        s += f_cython(a + i * dx)
    return s * dx

In [ ]:
%%timeit
integrate_f_cython3(0, 10, 1000000)

1.55 ms ± 12.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Таким образом, мы смогли без использования отдельных фреймворков и библиотек снизить время работы функции более чем в 100 раз!

Посмотрим, как справляется с той же задачей numpy.

In [ ]:
%%timeit
x = np.linspace(0, 10, 1000000)
(x**2).sum() / 100000

4.26 ms ± 97.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Таким образом, с помощью Cython можно написать более эффективный код по сравнению с numpy!

## Пример - работа с матрицами

Вернёмся к нашему примеру с Matrix:

In [ ]:
%%cython -a 
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        n_rows, n_cols = shape
        return cls([[0] * n_cols for i in range(n_rows)])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M

    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return ((0, 0) if not self else
                (int(len(self)), int(len(self[0]))))


def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()
    for i, Xi in enumerate(X):
        for k, Ytk in enumerate(Yt):
            Z[i][k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z

Пока всё желтое, т.е. cython не может распознать типы и обращается к интерпрератору python

In [ ]:
shape = (100, 100)

In [ ]:
X = Matrix.random(shape)
Y = Matrix.random(shape)

In [ ]:
%timeit cy_matrix_product(X, Y)

84.5 ms ± 1.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Получилось быстрее, чем наша питоновская реализация, но пока сильно не дотягивает до numpy версии.

Попробуем скрестить NumPy и Cython:


In [ ]:
X = np.random.randint(-255, 255, size=shape, dtype=np.int64)
Y = np.random.randint(-255, 255, size=shape, dtype=np.int64)

In [ ]:
%%cython -a
import numpy as np

def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [ ]:
%timeit cy_matrix_product(X, Y)

503 ms ± 11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Не помогло, всё стало ещё медленнее.

**Почему так?**

Дело в том, что Cython не умеет работать с питонячими импортами и теперь все строки для него непонятны и он обращается к интерпретатору питона.

Окей, давайте явно подскажем cython'у про типы и про импорты. Используем cdef, чтобы определить переменные C-шных типов:

In [ ]:
%%cython -a

import numpy as np
cimport numpy as np

def cy_matrix_product2(np.ndarray X, np.ndarray Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray Z
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [ ]:
%timeit  cy_matrix_product2(X, Y)

510 ms ± 14.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Вроде cython теперь больше понимает, но всё равно всё медленно, так как основные части всё ещё непонятны для Cython.

Добавим строгой типизации входных значений - теперь наша функция принимает только массивы определённого типа.

In [ ]:
%%cython -a
import numpy as np
cimport numpy as np  # Не любой модуль можно так импортировать, а только те, которые это поддерживают


def cy_matrix_product3(np.ndarray[np.int64_t, ndim=2] X,
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [ ]:
%timeit cy_matrix_product3(X, Y)

3.38 ms ± 66 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Ну теперь добавим совсем уже чёрной магии.<br>
Добавим декораторы, которые проверяют границы массивов и переполнение типов.

In [ ]:
%%cython 
import numpy as np

cimport cython
cimport numpy as np

@cython.boundscheck(False)
@cython.overflowcheck(False)
def cy_matrix_product4(np.ndarray[np.int64_t, ndim=2] X, 
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):        
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [ ]:
%timeit cy_matrix_product4(X, Y)

1.81 ms ± 8.54 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Profit.

# Extra. Numba

Указывать типы переменных вручную больно, по сути переписываем программу до состояния C-кода. Хочется, что-то более удобное!

In [ ]:
!pip install numba numpy==1.23.0


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


Импортируем несколько объектов:

In [ ]:
from numba import jit, njit, prange
import numpy as np

Рассмотрим простой пример - вычисление числа Пи с помощью метода Монте-Карло:

In [ ]:
import random

nsamples = 10_000_000
X = [random.random() for _ in range()]
Y = [random.random() for _ in range()]

In [ ]:
def monte_carlo_pi(X, Y):
    acc = 0
    for x, y in zip(X, Y):
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4 * acc / len(X)

In [ ]:
%%time
monte_carlo_pi(10_000_000)

CPU times: user 5.28 s, sys: 350 µs, total: 5.28 s
Wall time: 5.3 s


3.1411428

In [ ]:
@jit(nopython=True)
def monte_carlo_pi_nb(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4 * acc / nsamples

In [ ]:
%%time
monte_carlo_pi_nb(10_000_000)

CPU times: user 926 ms, sys: 3.17 ms, total: 929 ms
Wall time: 966 ms


3.1419128

In [ ]:
nsamples = 10_000_000
x = np.random.random(nsamples)
y = np.random.random(nsamples)

In [ ]:
def monte_carlo_pi_np(x, y):
    acc = np.sum(x ** 2 + y ** 2 < 1)
    return 4 * acc / len(x)

In [ ]:
%%time
monte_carlo_pi_np(x, y)

CPU times: user 77.4 ms, sys: 801 ms, total: 878 ms
Wall time: 905 ms


3.1422584

In [ ]:
@njit()
def monte_carlo_pi_np_nb(x, y):
    acc = np.sum(x ** 2 + y ** 2 < 1)
    return 4 * acc / len(x)

In [ ]:
%%time
monte_carlo_pi_np_nb(x, y)

CPU times: user 527 ms, sys: 3.38 ms, total: 531 ms
Wall time: 531 ms


3.1422584

Numba хорошо работает только с переменными NumPy-типов, поэтому возьмем базовую версию программы, но на вход подадим`ndarray`:

In [ ]:
@jit(nopython=True)
def matrix_product_nb(X, Y):
    n_xrows, n_xcols = len(X), len(X[0])
    n_yrows, n_ycols = len(Y), len(Y[0])    
    Z = [[0] * n_ycols for _ in range(n_xrows)]
    
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z 

Запустим и посмотрим:

In [ ]:
%%time
res = matrix_product_nb(X_test_np, Y_test_np)
res[0][0]

CPU times: user 863 ms, sys: 9.32 ms, total: 873 ms
Wall time: 874 ms


-43538

Получилось не особо впечатляюще. Попробуем запустить еще раз:

In [ ]:
%%time
res = matrix_product_nb(X_test_np, Y_test_np)
res[0][0]

CPU times: user 144 ms, sys: 72 µs, total: 144 ms
Wall time: 143 ms


-43538

А вот это уже результат!

Еще одна удобная фишка Numba - удобная параллелизация вычислений. Для этого нам пригодится функция `prange`:

In [ ]:
@njit(parallel=True)
def matrix_product_nb_parallel(X, Y):
    n_xrows, n_xcols = len(X), len(X[0])
    n_yrows, n_ycols = len(Y), len(Y[0])    
    Z = [[0] * n_ycols for _ in range(n_xrows)]
    
    for i in prange(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z 

In [ ]:
%%time
res = matrix_product_nb_parallel(X_test_np, Y_test_np)
res[0][0]

CPU times: user 2.27 s, sys: 5.73 ms, total: 2.28 s
Wall time: 1.99 s


-43538

In [ ]:
%%time
res = matrix_product_nb_parallel(X_test_np, Y_test_np)
res[0][0]

CPU times: user 486 ms, sys: 5.56 ms, total: 491 ms
Wall time: 51.9 ms


-43538

# Extra. Memory Usage (just in case)

Помимо быстродействия вашей программы важно следить и за потребляемой памятью. Для этого пригодится библиотека `memory_profiler`:

In [ ]:
!pip install memory_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for memory-profiler: filename=memory_profiler-0.60.0-py3-none-any.whl size=31285 sha256=b13690d49aabec22a9c553f7220b67cb861b90011ecd02b49eabeee3fd5937c2
  Stored in directory: /root/.cache/pip/wheels/67/2b/fb/326e30d638c538e69a5eb0aa47f4223d979f502bbdb403950f
Successfully built memory-profiler


In [ ]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


## memit

In [ ]:
%%memit
a = [0] * 2*10**8
del a

peak memory: 1940.37 MiB, increment: 567.73 MiB


## mprun

Хотим более подробное профилирования потребляемой памяти - используем `mprun`:

### jupyter mprun

Увы, в Jupyter (Colab)-ноутбуках сразу выполнить профайлинг не получится:

In [ ]:
def my_func():
    a = [1] * 1000000
    b = [2] * 9000000
    del b
    return a

%mprun -T mprof0 -f my_func my_func()


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 845, in enable
    sys.settrace(self.trace_memory_usage)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 848, in disable
    sys.settrace(self._original_trace_function)



ERROR: Could not find file <ipython-input-79-e8b08a7c72f4>
NOTE: %mprun can only be used on functions defined in physical files, and not in the IPython environment.


*** Profile printout saved to text file mprof0. 


Для этого сначала сохраним наш код в файл:

In [ ]:
%%writefile memscript.py
def my_func():
    a = [1] * 1000000
    b = [2] * 9000000
    del b
    return a

Writing memscript.py


Затем импортируем и запустим профилирование:

In [ ]:
from memscript import my_func
%mprun -T mprof0 -f my_func my_func()



*** Profile printout saved to text file mprof0. 


In [ ]:
!cat mprof0

Filename: /content/memscript.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     1   1940.4 MiB   1940.4 MiB           1   def my_func():
     2   1940.4 MiB      0.0 MiB           1       a = [1] * 1000000
     3   1940.4 MiB      0.0 MiB           1       b = [2] * 9000000
     4   1940.4 MiB      0.0 MiB           1       del b
     5   1940.4 MiB      0.0 MiB           1       return a

### Консольный mprun

Если имеем написанную программу, то необязательно профилировать ее из ноутбука:

In [ ]:
%%writefile memscript2.py
@profile
def my_func():
    a = [1] * 1000000
    b = [2] * 9000000
    del b
    return a

my_func()

Writing memscript2.py


In [ ]:
!cat memscript2.py

@profile
def my_func():
    a = [1] * 1000000
    b = [2] * 9000000
    del b
    return a

my_func()


Запускаем модуль `memory_profiler` прямо из консоли:

In [ ]:
!python -m memory_profiler memscript2

Filename: /content/memscript2.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     1   37.715 MiB   37.715 MiB           1   @profile
     2                                         def my_func():
     3   45.449 MiB    7.734 MiB           1       a = [1] * 1000000
     4  114.285 MiB   68.836 MiB           1       b = [2] * 9000000
     5  114.285 MiB    0.000 MiB           1       del b
     6  114.285 MiB    0.000 MiB           1       return a


